# Playing with filtering (deprecated?)

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import scipy.optimize
import scipy.signal as signal
from scipy.integrate import cumtrapz

import gm2
import trfp
import root_to_pandas
import helper_function_candidates as helper
import plotting_functions as plt2

import allantools

## Import tier-1 data from ROOT tree

In [ ]:
%%time
fp_interp_df = root_to_pandas.root_to_pandas(range(3960,3990), prefix='v9_21_03_dev/FieldPlainRootOutput_', tr_run=False)
fp_moment_df = root_to_pandas.calc_moment_df(fp_interp_df)

# Everything below here is after initial run loading

In [ ]:
interp_df.head()
plt.plot(interp_df['fp3'])

In [ ]:
taus, adev, _, _ = allantools.mdev(interp_df['fp0'].values, data_type='freq', rate=1, taus='decade')
plt.loglog(taus, adev, '.')
taus, adev, _, _ = allantools.mdev(interp_df['fp3'].values, data_type='freq', rate=1, taus='decade')
plt.loglog(taus, adev, '.')

## Need to define good, ok, and bad probes for filter

This should come from looking at Allan deviations. In general, the bad probes have $adev(\tau=1) > 10$, good probes have $adev(\tau=1) < 1$, and ok probes form the middle ground. 

In [ ]:
# fp_bad = [3,4,5,9,10,11,15,16,17,21,22,23,27,28,33,34,35,
#           74,75,124,221,257,332,341,342,372]

# fp_ok = []

# fp_good = range(378)
# for bad in fp_bad:
#     fp_good.remove(bad)
# for ok in fp_ok:
#     fp_good.remove(ok)

# fp_bad = []
# fp_ok = []
# fp_good = []

# for fp in range(378):
#     # look at adev of tau = 1
#     if np.sqrt(0.5 * np.mean(np.diff(fp_freq_interp[:,fp])**2)) > 10.0:
#         fp_bad.append(fp)
#     elif np.sqrt(0.5 * np.mean(np.diff(fp_freq_interp[:,fp])**2)) < 1.0:
#         fp_good.append(fp)
#     else:
#         fp_ok.append(fp)

# print fp_bad
# print fp_ok
# print fp_good

fp_bad = [3, 4, 5, 9, 11, 15, 16, 17, 21, 22, 23, 27, 28, 33, 34, 35, 74, 257]

## Filter fixed probes based on the lists above
Include filtering in trolley footprint removal

In [ ]:
# filter the bad probes with 3rd order Butterworth
tau = 200.
b, a = signal.butter(3, 1./tau)

fp_freq_filt = grid_freqs.copy()

for fp in fp_bad:
    # use "Gustafson’s method", which makes initial conditions such that
    # forward-backward filtering gives same results as backward-forward
    fp_freq_filt[:, fp] = signal.filtfilt(b, a, fp_freq_filt[:, fp], method='gust')

In [ ]:
print np.reshape(qual[0,:],[378/3,3])

In [ ]:
print freq2.shape
print np.max(np.max(qual))

In [ ]:
dropped = np.empty(378)
for pr in range(378):
    dropped[pr] = float(np.count_nonzero(np.isnan(freq2[:,pr])))/float(freq2[:,pr].size)

    
print np.max(dropped)
plt.hist(dropped, bins = 50)

In [ ]:
times = np.arange(np.ceil(time[0, 377]),
                  np.floor(time[-1, 0]) + 1,
                  1)  # NOTE THIS IS 1 SECOND NOW, DOESN'T MATTER BECAUSE OF INTEGRATION METHOD

interp = np.empty((times.size, 378))
for pr in range(378):
    not_nan = ~np.isnan(freq2[:,pr])
    interp[:,pr] = np.interp(times,time[:,pr][not_nan],freq2[:,pr][not_nan])

In [ ]:
probe = 3

fig, axs = plt.subplots(4,1)

plt.sca(axs[0])
plt.plot(time[:,probe], freq[:,probe], '.')

# axs[0].set_xlim([1525283150,1525283350])

plt.sca(axs[1])
plt.plot(time[:,probe], qual[:,probe], '.')
# axs[1].set_xlim([1525283150,1525283350])

plt.sca(axs[2])
plt.plot(time[:,probe], freq2[:,probe], '.')

# axs[2].set_xlim([1525283150,1525283350])

plt.sca(axs[3])
plt.plot(times, interp[:,probe], '.')

fig.set_size_inches(12,12)
fig.tight_layout()

In [ ]:
# try filtering a super noisy probe
from scipy import signal
import allantools

probe = 100

taus, adev, _, _ = allantools.mdev(interp[:,probe], data_type='freq', rate=1, taus='all')

b, a = signal.butter(1,1./10.)
y = signal.filtfilt(b, a, interp[:,probe])

taus2, adev2, _, _ = allantools.mdev(y, data_type='freq', rate=1, taus='all')

fig, axs = plt.subplots(4,1)

plt.sca(axs[0])
plt.plot(times, interp[:,probe], '.')

# axs[0].set_xlim([1525283150,1525283350])

plt.sca(axs[1])
plt.loglog(taus, adev, '.')

plt.sca(axs[2])
plt.plot(times, y, '.')
# axs[1].set_xlim([1525283150,1525283350])

plt.sca(axs[3])
plt.loglog(taus2, adev2, '.')

fig.set_size_inches(12,12)
fig.tight_layout()

In [ ]:
start = 0 * 42  # steps of 42

fig, axs = plt.subplots(126,3)

for i in range(126):
    for j in range(3):
        plt.sca(axs[i,j])
        taus, adev, _, _ = allantools.mdev(interp[:,start + 3*i + j], data_type='freq', rate=1, taus='octave')
        plt.loglog(taus, adev, ':.', markersize=3)
        axs[i,j].set_title('pr'+ str(start + 3*i + j))

fig.set_size_inches(15,240)
fig.tight_layout()

# fig.savefig('all_allan_4141.png', dpi=100)

In [ ]:
pr = 100

print np.mean(interp2[:,pr] - interp[:,pr])
print np.std(interp2[:,pr] - interp[:,pr])

fig, ax = plt.subplots(1,1)
plt.plot(times, interp2[:,pr] - interp[:,pr], '.')

fig.set_size_inches(12,4)
fig.tight_layout()